In [1]:
import music21 as m21
import re
from collections import Counter
from swalign_local import *

In [548]:
# Step 1: Read input data:
import music21 as m21
import re
from collections import Counter
from swalign_local import *
import csv
import glob

#score_path = "jkupddtest/bachmono/wtc2f20.krn"
#score_path = "jkupddtest/beethovenmono/sonata01-3.krn"
#score_path = "jkupddtest/mozartmono/sonata04-2.krn"
#score_path = "jkupddtest/gibbonsmono/silverswan.krn"
score_path = "jkupddtest/chopinmono/mazurka24-4.krn"
#score_path = '~/facets-search-engine/data/Beethoven9thOdeToJoy.xml'

raw_score = m21.converter.parse(score_path)
num_of_parts = len(raw_score.parts)

In [549]:
num_of_parts

2

In [550]:
voices = []
for part in raw_score.parts:
    voices.append(part)

In [551]:
# Step 2: Get diatonic scale degree sequence from all voices

def get_diatonic_intervals(voice):
    
    # get key / root information:
    roots = [key.tonic for key in voice.recurse().getElementsByClass(m21.key.Key)]
    diatonic_root = roots[0].diatonicNoteNum if roots != [] else voice.analyze('key').tonic.diatonicNoteNum
    #print(diatonic_root)
    key = voice.analyze('key')

    noteoffset = []
    midi = []
    #timesig = m21_score.getContextByClass(m21.meter.TimeSignature)
    #barlength = timesig.barduration

    dict_wordtonum = {"Unison": '2', "Second": '2', "Third": '3', "Fourth": '4', "Fifth": '5', "Sixth": '6', "Seventh": '7'}
    #P.S.: Unison as "2" instead of "0" for a reason, see explanation later
        
    # encoding the diatonic intervals. not 1A and 1D are represented as 2A and 2D here...
    dict_encode_dia_intervals = {"7D": 'A', '6D': 'B', '5D': 'C', '4D':'D', '3D':'E', '2D':'F', 
                                     '2A':'G', '3A':'H', '4A':'I', '5A':'J', '6A': 'K', '7A':'L', '0A': 'M'}
        
    diatonic_intervals = []
    beatstrength = []
    previous_note = None

    onset = 0
    for thisnote in voice.recurse().notes:
        onset += thisnote.offset
        
        # We ignore rests
        if thisnote.isRest: 
                # If the rest is a full measure, part of a multi-measure rest: we need to adjust
                continue
        
        noteoffset.append(onset)
        beatstrength.append(thisnote.beatStrength)
        
        if thisnote.isNote:
            midi.append(thisnote.pitch.midi)
        elif thisnote.isChord:
            midi.append(thisnote.root().midi)
            
        if previous_note is None:
                previous_note = thisnote
        else:
                if thisnote.isNote and previous_note.isNote:
                    # gap = number of semi-tones of the current interval 
                    gap = thisnote.pitch.diatonicNoteNum - previous_note.pitch.diatonicNoteNum
                elif thisnote.isChord and previous_note.isNote:
                    gap = thisnote.root().diatonicNoteNum - previous_note.pitch.diatonicNoteNum
                elif thisnote.isChord and previous_note.isChord:
                    gap = thisnote.root().diatonicNoteNum - previous_note.root().diatonicNoteNum
                elif thisnote.isNote and previous_note.isChord:
                    gap = thisnote.pitch.diatonicNoteNum - previous_note.root().diatonicNoteNum
                # if a pitch change is detected
                    
                if gap != 0:
                    if gap > 0:
                        #  if the semi-tone difference between the current and the previous item > 0, it is an ascending interval.
                        direction = 'A'
                    else:
                        #  otherwise, it is a descending interval.
                        direction = 'D'

                    # Get intervals using music21
                    """
                            "directedSimpleNiceName" examples: "Descending Doubly-Diminished Fifth", "Ascending Perfect Fourth", "Ascending Doubly-Augmented Fourth"
                            "simpleName" examples: dd5, P5, AA4. There's no direction information
                            Since it only executes when a pitch interval is detected, "unison" refers to an augmented unison, a.k.a minor second
                    """
                    # take intervals between root notes if there exists any chord
                    if previous_note.isChord:
                        startnote = previous_note.root()
                    else:
                        startnote = previous_note
                    if thisnote.isChord:
                        endnote = thisnote.root()
                    else:
                        endnote = thisnote
                        
                    m21_interval_directed = m21.interval.Interval(noteStart=startnote, noteEnd=endnote).directedSimpleNiceName

                    arr_diatonic = m21_interval_directed.split(" ")

                    m21_generic = dict_wordtonum[arr_diatonic[-1]]
                    
                    # m21_interval: 2A, 3D etc...
                    m21_interval = m21_generic+direction
                    # to make each m21_interval unique, show direction and each as single character in string, we encode the diatonic intervals as letters
                    encode_interval = dict_encode_dia_intervals[m21_interval]
                    diatonic_intervals.append(encode_interval)

                else:
                    # We take the interval between two consecutive pitches as 0A
                    encode_interval = dict_encode_dia_intervals['0A']
                    diatonic_intervals.append(encode_interval)
                previous_note = thisnote
            
    string = ""
    for i in diatonic_intervals:
        string += str(i)
        
    return string, key, noteoffset, midi, beatstrength

In [552]:
strings = []
noteoffsets = []
midis = []
keys = []
beatstrengths = []

for voice in voices:
    string, key, noteoffset, midi, beatstrength = get_diatonic_intervals(voice)
    strings.append(string)
    noteoffsets.append(noteoffset)
    beatstrengths.append(beatstrength)
    midis.append(midi)
    keys.append(key)
    print(len(string), len(midi))
    print(string)
    print(midi)
    print(beatstrength)

1049 1050
GMFMMLMAMMLMAGMJMCMMJMCGMIDGMMJMCGMIDGMMJMCGMIDGMMMGMGFHFFFJDIDEGFFIGMMCFMILDIDEGFFIGMMCFMILDIDEGFFIGMCFMKHFMAGMFBKJDDEHGCFBKLHMFBKAMLAMLAMKBMKAMLAMKAMLAMLAMKAMLFGFFFDMFJFBKBMHDDEHGCFBKLHMFBKAMLAMLAMKBMKAMLAMKAMKBMKAMKBMKAMLAMLMDIDDHFFIGMMCFMILDIDEGFFIGMMCFMILDIDEGFFIGMCFMKMGMAHFMFFJDIDEGFGFFIGMMGFCFMILDIDEGFGFFIGMMGFCFMILDIDEGFGFFIGMCFMKHFMAGMFBKJDDEHGCFBKLHMFBKAMLAMLAMKBMKAMLAMKAMLAMLAMKAMLFGFFFDMFJFBKBMHDDEHGCFBKLHMFBKAMLAMLAMKBMKAMLAMKAMKBMKAMKBMKAMLAMLMDIDDHFFIGMMCFMILDIDEGFFIGMMCFMILDIDEGFFIGMCFMKMGMAHFMFFJDIDEGFGFFIGMMGFCFMILDIDEGFGFFIGMMGFCFMILDIDEGFGFFIGMCFMKHFMAGJMGDGEFMJHFMDGEFCGMJFJMFGMHFFFCKGDGEFMJHFMDGEFCGMJFJMFGMHFFMFCKGFFGGGHFFLMGFFFGEHGFFGGGHFFLMGFFFGEIGFFGGGHFFLMGFFFGEGGFFGGGHFFLMGFFFJFEGFFGGGHFFLMGFFFGEHGFFGGGHFFLMGFFFGEIGFFGGGHFFLMGFFFGEGFGGGHFFLMGFFFCLMIFDFCJMGFFFDGGFALBJFHBMKBMKBMKBMKAMKAMKBMKAMKBMKAMLAMLMDIDDHFFIGMMCFMILDIDEGFFIGMMCFMILDIDEGFFIGMMCFMKGMAHFMFFJDIDEGFGFFIGMMGFCFMILDIDEGFGFFIGMMGFCFMILDIDEGFGFFIGMMCFMGFMAGMGMMMGMMFMFMEMIFMEMFMJFMCMFMKGMMMG

521 522
GMCHACGMCHACLAKMEFHHFHMEJMCGCCGMCGCCGMIFMFHECHACGMCHKDGMCHEGGBGIMDGMIFCGGMCHKDGMCHEGGBGIMDHCGMCGCCGMCGCCGMIHEFHHFHMEJMCGCCGMCGCCGMCMGFHECHACGMCHKDGMCHEGGBGIMDGMIFCGGMCHKDGMCHEGGBGIMDHCGMCGCCGMCGCCGMIHEFHHFHMEJMCGCCGMCGCCGMCMGFHECHHMGDGEFMJHFMDGEFAIMDIMDIMDIGGDGEFMJHFMDGEFAIMDIMDIMDIAKMBKMCHMAHMAKMBKMCHMAHMBKMBKMCHMAHMFKMBKMCHMEHMEHMEHMEHMAHMAKMBKMCHMAHMBKMBKMCHMAHMFKMBKMCHEHEGFHEHCGMCGCCGMCGCCGMIHEFHHFHMEJMCGCCGMCGCCGMIFMFHECHMEIMDIMDGMFKMBGMGMEMGMFMFMIMDIMDGMFKMBIMDHMEJMCGMFJMMMMCIMDGMFJMMMMCKMMMIFHMMMMBGMEGMEGMMMM
[41, 53, 53, 46, 61, 51, 44, 56, 56, 49, 65, 55, 48, 70, 60, 68, 68, 53, 41, 69, 72, 46, 61, 61, 46, 53, 53, 46, 58, 51, 44, 56, 56, 49, 61, 55, 48, 60, 60, 65, 53, 53, 41, 69, 53, 46, 61, 51, 44, 56, 56, 37, 53, 61, 44, 56, 56, 49, 65, 50, 51, 63, 42, 44, 61, 61, 44, 56, 56, 61, 49, 43, 44, 56, 56, 37, 53, 61, 44, 56, 56, 49, 65, 50, 51, 63, 42, 44, 61, 61, 44, 60, 41, 53, 53, 46, 58, 51, 44, 56, 56, 49, 61, 55, 48, 60, 60, 65, 69, 53, 41, 69, 72, 46, 61, 61, 46, 5

In [523]:
# Smith-Waterman alignment between the two input sequences
def swm_alignment(string1, string2):
    match = 2
    mismatch = -2
    scoring = NucleotideScoringMatrix(match, mismatch)
    sw = LocalAlignment(scoring)#, gap_extension_penalty = -5) 
    
    alignment = sw.align(string1, string2)
    alignment_strings = alignment.dump()

    simi_score = alignment.matches / (alignment.mismatches + alignment.matches)
    
    return alignment_strings, simi_score


In [524]:
def filter_and_gather_patterns(allthepatterns, fullseq):
    pat = fullseq.split('-')
    print("num of pat in this seq:", len(pat))    
    for pattern in pat:
        # Get rid of the sequence shorter than 4; combine all alignment patterns into all pattern list
        if len(pattern) > 2:
            allthepatterns.append(pattern)
    return allthepatterns

In [525]:
# compare each pair of voices!
allthepatterns = []

for i in range(0, len(voices)-1):
    for j in range(i+1, len(voices)):
        alignment_strings, simi_score = swm_alignment(strings[i], strings[j])
        if simi_score < 0.25:
            print("ignoring voice pair:", i, j)
            # if two voices are very different, getting patterns from their alignment won't make sense
            continue
        # get the patterns from the first voice of this pair
        fullseq1 = alignment_strings[0]
        fullseq2 = alignment_strings[1]
        allthepatterns = filter_and_gather_patterns(allthepatterns, fullseq1)
        allthepatterns = filter_and_gather_patterns(allthepatterns, fullseq2)
        

Query:   2 IAJEIEEEKEEEKE--------IFAGFFFFHFFFFGGMFFFFGGMFFFFGGMFGG----MGGGFFFFM-GGGFFFFGMMFGGGFFFFMMGGGFFFFDIIHCHCHHEICEIE--MGMFGGLFJMCIFFFFH----FFFFGGMFFFFGGMFFFFGGMFGGFEKEMGGGFFFFHCH--FIAJEIEKFFFFHHHFEFFGGCLFGGGMFGE------CHCHCMMCHCHCMDE----K------------GGFFFFFFGEF-FGJFFFFFFKFFFFFFF--GJGGFFGG 238
           ||||||||||||||        .|.||||||||||||||||||||||||||||||    |  |    | |||    |   |||       ||   |  |    ..|| |.|  .|  || |     ||||    ||||    ||| | ||||  .|.||   | ...|..|||  .|      ||||||||||||||||     .|      .|            |    |            ||||||||  | |  ||||||.|||||||  |  |.|||
Ref  :   2 IAJEIEEEKEEEKEGGMMGGGFFFFGFFFFHFFFFGGMFFFFGGMFFFFGGMFGGDEIEM--G----MFGGG----G---GGG-------GG---F--I----AJEI-EEEKEEG--GG-F-----FFFHGGGGFFFF----FFF-G-FFFF--FFFGF---M-CIMFIMHCHCHMI------FFFFHHHFEFFGGCLF-----IEIAJEIEEH------------EIEEEKEEEKEMFGGGGGGGFFFFFF--FIF--FFFFFFIFFFFFFFKFG--GMFGG 205

num of pat in this seq: 47
num of pat in this seq: 80
Query:   1 EIAJEIEEEK-----AG-----FFFF-----HFFFFGGMFF

In [526]:
allthepattern_count = dict(Counter(allthepatterns))
# elimiate the repeated strings

allthepatterns = list(set(allthepatterns))

In [527]:
def count_substringandparents(allthepatterns, allthepattern_count):
    
    issubstring = {}
    hassubstring = {}
    count_as_substring = {}
    count_as_parent = {}

    for i in range(0, len(allthepatterns)-1):
        for j in range(i+1, len(allthepatterns)):
            if allthepatterns[i] in allthepatterns[j]:
                parent = allthepatterns[j]
                kid = allthepatterns[i]
                times = parent.count(kid)

                if kid not in issubstring:
                    issubstring[kid] = 1
                    count_as_substring[kid] = times * allthepattern_count[parent]
                else:
                    issubstring[kid] += 1
                    count_as_substring[kid] += times * allthepattern_count[parent]

                if parent not in hassubstring:
                    hassubstring[parent] = 1
                    count_as_parent[parent] = 1#times
                else:
                    hassubstring[parent] += 1
                    count_as_parent[parent] += 1 

            elif allthepatterns[j] in allthepatterns[i]:
                parent = allthepatterns[i]
                kid = allthepatterns[j]
                times = parent.count(kid)

                if kid not in issubstring:
                    issubstring[kid] =1
                    count_as_substring[kid] = times * allthepattern_count[parent]
                else:
                    issubstring[kid] += 1
                    count_as_substring[kid] += times * allthepattern_count[parent]

                if parent not in hassubstring:
                    hassubstring[parent] = 1
                    count_as_parent[parent] = 1 #times
                else:
                    hassubstring[parent] += 1
                    count_as_parent[parent] += 1 #times

    return issubstring, hassubstring, count_as_substring, count_as_parent


In [528]:
issubstring, hassubstring, count_as_substring, count_as_parent = count_substringandparents(allthepatterns, allthepattern_count)

In [529]:
# rank the ones that are substrings of another detected pattern
sortis = dict(Counter(issubstring))
sorthas = dict(Counter(hassubstring))
print(len(sorthas))
print(len(sortis))

56
42


In [530]:
# This counts the times each pattern appeared after original segmentation plus the times they are in other patterns,
# plus the times other patterns showed up in them.

def combined_count(sorthas, sortis, count_as_parent, count_as_substring, allthepattern_count):
    refined_combined = {}

    for item in sorthas:
        if item in sortis:
            # the ones that has substring and also are the substrings
            refined_combined[item] = count_as_parent[item] + count_as_substring[item] + allthepattern_count[item]
        else:
            # the ones that has substrings
            refined_combined[item] = count_as_parent[item] + allthepattern_count[item]
    # the patterns that are substrings of other patterns but none of the others are substrings of it
    for item in sortis:
        if item not in refined_combined:
            refined_combined[item] = count_as_substring[item] + allthepattern_count[item]
        # otherwise is already counted
    
    return refined_combined

combined = combined_count(sorthas, sortis, count_as_parent, count_as_substring, allthepattern_count)

In [531]:
#dict(Counter(combined))

In [532]:
def filter_combined_patterns(combined):
    
    finalpattern = []
    for i in combined: 

        if len(i) < 3:
            # if the length is shorter than 4 notes, discard
            continue

        if combined[i] > 1:
            # keep the ones that has substring or is substring more than once
            finalpattern.append(i)

    finalpattern_combined = {}
        
    for thisone in finalpattern:
        finalpattern_combined[thisone] = combined[thisone]

    dict(Counter(finalpattern_combined))
        
    if len(finalpattern) > 20:
        # if more than 20 patterns, only take the top 20
        finalpattern_combined = dict(sorted(finalpattern_combined.items(), key = lambda x:-x[1], reverse = True)[-20:])
    
    return finalpattern_combined


In [533]:
# filter out the short ones
top20patterns_combined = filter_combined_patterns(combined)

In [534]:
print(dict(Counter(top20patterns_combined)))

{'AJEI': 14, 'FFFGGMFFFF': 15, 'AGFFFFHFFFFGGMFFFFGGMFFFFGGMFGGMGGGFFFFMGGGFFFFG': 16, 'HFFFFGGMFFFFGGMFFFFGGMFGG': 16, 'MGGGFFFFMGGGFFFFGGGFFGGMCHCHICKGEGGGFFEIFFIAJEIEMGMFGGEICEIEEEKAKCHCHICFGGGFFFFJEKCEIMMGGGFFFFGFFFFFFFGFFFFHFFFFGGM': 17, 'FGGG': 18, 'GGFFFFJEKCEIMMGGGFFFFGFFFFFFFGFFFFHFFFFGGMFFFFGGMFFFFGGMFGGGGMF': 19, 'GGMFGG': 19, 'IAJEIEEEKEEEKEGGMMGGGFFFFGFFFFHFFFFGGMFFFFGGMFFFFGGMFGGDEIEM': 21, 'EIAJEIEEEKEEEKEGGMMGGGFFFFGFFFFHFFFFGGMFFFFGGMFFFFGGMFGGDEIEM': 22, 'GMFGG': 23, 'IAJEIEEEKEEEKEIFAGFFFFHFFFFGGMFFFFGGMFFFFGGMFGGMGGGFFFFMGGGFFFFGMMFGGGFFFFM': 24, 'GGGFFFF': 26, 'GGFF': 33, 'FFFGGMF': 35, 'GGG': 46, 'FFGG': 46, 'GMF': 52, 'FFFF': 85, 'FFF': 103}


In [535]:
def reduce_pattern_length(seq):
        # the function returns the longest substring that appeared at least once
        # if there is length 4 substring repeated 4 times and a length 7 substring repeated 2 times, we take the length 7
        best_performance = ""
        candidates = []
        for length in range(int(0.3*len(seq)), int(len(seq)*0.5)+1): 
            for start in range(0, len(seq)-length):
                # get all the substrings of this length within the string, save in candidates
                candidates.append(seq[start:start+length])
        count_can = {}
        for candidate in candidates:
                count_can[candidate] = seq.count(candidate)
                if count_can[candidate] > 1:
                    # if it is repeated more than once in the string
                    if len(candidate) > len(best_performance):
                        best_performance = candidate
                    elif len(candidate) == len(best_performance) and count_can[candidate] > count_can[best_performance]:
                        best_performance = candidate
        
        return best_performance

def check_reduction(item):
    success = False
    frequent_substr = reduce_pattern_length(item)
    if len(frequent_substr) > 4 and len(frequent_substr) > 0.3*len(item):
        print("extracted", frequent_substr, "from", item)
        # sucessfully reduced the pattern
        return frequent_substr
    return None

In [536]:
def reduce_length_of_each_pattern(onlyassubstring, top20patterns):
    
    beforereduction = {}
    # reducedfinalpat is the new list that contains all valid patterns after reduction plus the ones that does not need to be reduced
    reducedfinalpat = []

    for item in top20patterns:
        # check the ones that has substring, see if their length can be reduced
        if len(item) > 11 and item not in onlyassubstring:
            #if the extracted pattern is too long, find longest repeated pattern within it
            frequent_substr = check_reduction(item)
            if frequent_substr != None:
                if len(frequent_substr) > 23:
                    # second attempt, if a frequent substring is takend and it is still quite long
                    new_frequent_substr = check_reduction(frequent_substr)
                    if new_frequent_substr != None:
                        # if the second attempt is a success, take the twice reduced pattern
                        reducedfinalpat.append(new_frequent_substr)
                        beforereduction[new_frequent_substr] = item
                    else:
                        # if the second attempt failed, take the result of first reduction
                        reducedfinalpat.append(frequent_substr)
                        beforereduction[frequent_substr] = item
                else:
                    # if the extracted pattern is already not very long
                    reducedfinalpat.append(frequent_substr)
                    beforereduction[frequent_substr] = item
            else:
                # if the reduction failed, take the original
                reducedfinalpat.append(item)
        else:
            # otherwise keep it without reduction
            reducedfinalpat.append(item)
    
    return reducedfinalpat, beforereduction

In [537]:
onlyassubstring = set(issubstring) - set(hassubstring)
reducedfinalpat, beforereduction = reduce_length_of_each_pattern(onlyassubstring, list(set(top20patterns_combined)))

extracted FFFFGGMF from HFFFFGGMFFFFGGMFFFFGGMFGG


In [538]:
def find_pattern_pos(string, pattern, beatstrength):
    patpos_start = []
    patpos_end = []
    count = 0
    beatcount = 0
    for m in re.finditer(pattern, string):
        count +=1
        if beatstrength[m.start()] == 1: # or >= 0.5
            beatcount+=1
        patpos_start.append(m.start())
        patpos_end.append(m.end())
    return patpos_start, patpos_end, count, beatcount

In [539]:
def count_final_pattern(reducedfinalpat, strings, beatstrengths):

    count_pattern = {}
    refined_finalpat = []
    startonsets = []
    startpitches = []
    endonsets = []
    endpitches = []

    for pattern in reducedfinalpat:
        stringnum = 0
        beat_importance = 0
        for string in strings:
            # find the pattern in eachvoice
            patpos_start, patpos_end, count, beatcount = find_pattern_pos(string, pattern, beatstrengths[stringnum])
            beat_importance += beatcount
            if pattern not in count_pattern:
                count_pattern[pattern] = count
            else:
                count_pattern[pattern] += count
            if count != 0:
                print("Pattern", pattern, "appears in voice no.", stringnum+1, ", first pos starts at ", patpos_start[0], "end at", patpos_end[0])
            num = 0
            for startpos in patpos_start:
                startonsets.append(noteoffsets[stringnum][startpos])
                startpitches.append(midis[stringnum][startpos])
                endonsets.append(noteoffsets[stringnum][patpos_end[num]])
                endpitches.append(midis[stringnum][patpos_end[num]])
                num+=1
            stringnum+=1
        if count_pattern[pattern] >= 2 and beat_importance > 0:
            # get rid of the ones that actually appeared less than 2 times in total
            refined_finalpat.append(pattern)
        else:
            if count_pattern[pattern] < 2:
                print("discard pattern", pattern, "because it only appered once in total.")
            elif beat_importance == 0:
                print("discard pattern", pattern, "because it never appeared at the start of a bar.")
                print("this pattern can be decoded as", decode_strings(pattern))
            
    return refined_finalpat, count_pattern, startonsets, startpitches, endonsets, endpitches


In [540]:
finalpat, count_pattern, startonsets, startpitches, endonsets, endpitches = count_final_pattern(reducedfinalpat, strings, beatstrengths)

Pattern GMFGG appears in voice no. 1 , first pos starts at  51 end at 56
Pattern GMFGG appears in voice no. 2 , first pos starts at  43 end at 48
Pattern GMFGG appears in voice no. 3 , first pos starts at  36 end at 41
discard pattern GMFGG because it never appeared at the start of a bar.
this pattern can be decoded as 2A0A2D2A2A
Pattern GGMFGG appears in voice no. 1 , first pos starts at  50 end at 56
Pattern GGMFGG appears in voice no. 2 , first pos starts at  42 end at 48
Pattern GGMFGG appears in voice no. 3 , first pos starts at  35 end at 41
Pattern GGFFFFJEKCEIMMGGGFFFFGFFFFFFFGFFFFHFFFFGGMFFFFGGMFFFFGGMFGGGGMF appears in voice no. 3 , first pos starts at  114 end at 177
discard pattern GGFFFFJEKCEIMMGGGFFFFGFFFFFFFGFFFFHFFFFGGMFFFFGGMFFFFGGMFGGGGMF because it only appered once in total.
Pattern FFFFGGMF appears in voice no. 1 , first pos starts at  32 end at 40
Pattern FFFFGGMF appears in voice no. 2 , first pos starts at  24 end at 32
Pattern FFFFGGMF appears in voice no. 3 , 

In [541]:
def decode_results(finalpat, count_pattern, combined, beforereduction):
    
    decoded_patterns = []
    decoded_patterns_weighed = {}
    decoded_patterns_count = {}
    decoded_patterns_combined = {}
    
    decode_dia_intervals = {'A': '7D', 'B': '6D', 'C': '5D', 'D': '4D', 'E': '3D', 'F': '2D', 
                            'G': '2A', 'H': '3A', 'I': '4A', 'J': '5A', 'K': '6A', 'L': '7A', 'M': '0A'}

    # decode all the candidates for patterns
    for pattern in finalpat:
        trans_pattern = ""
        for letter in pattern:
            trans = decode_dia_intervals[letter]
            trans_pattern += trans

        decoded_patterns_count[trans_pattern] = count_pattern[pattern]
        
        if pattern in combined:
            # some might be reduced to a shorter pattern
            decoded_patterns_combined[trans_pattern] = combined[pattern]
        else:
            decoded_patterns_combined[trans_pattern] = combined[beforereduction[pattern]]
        decoded_patterns.append(trans_pattern)
        
    return decoded_patterns, decoded_patterns_combined, decoded_patterns_count


In [542]:
def decode_strings(string):
    
    decode_dia_intervals = {'A': '7D', 'B': '6D', 'C': '5D', 'D': '4D', 'E': '3D', 'F': '2D', 
                            'G': '2A', 'H': '3A', 'I': '4A', 'J': '5A', 'K': '6A', 'L': '7A', 'M': '0A'}
    
    decode_string = ""

    for letter in string:
        trans = decode_dia_intervals[letter]
        decode_string+=trans

    return decode_string

In [543]:
decoded_strings = []
for string in strings:
    decoded = decode_strings(string)
    decoded_strings.append(decoded)
    print(decoded)
    print("")

3D4A7D5A3D4A3D3D3D6A3D3D3D6A3D2A2A0A0A2A2A2A2D2D2D2D2A2D2D2D2D3A2D2D2D2D2A2A0A2D2D2D2D2A2A0A2D2D2D2D2A2A0A2D2A2A4D3D4A3D0A2A0A2D2A2A2A2A2A2A2A2A2A2D4A7D5A3D4A3D3D3D6A3D3D2A2A2A2D2D2D2D3A2A2A2A2A2D2D2D2D2D2D2D2A2D2D2D2D2D2D2D2A2D0A5D4A0A2D4A0A3A5D3A5D3A0A4A2D2D2D2D3A3A3A2D3D2D2D2A2A5D7A2D4A3D4A7D5A3D4A3D3D3A3D4A3D3D3D6A3D3D3D6A3D0A2D2A2A2A2A2A2A2A2D2D2D2D2D2D2D4A2D2D2D2D2D2D2D4A2D2D2D2D2D2D2D6A2D2A2A0A2D2A2A0A2A5D3A2A

2D4A7D5A3D4A3D3D3D6A3D3D3D6A3D4A2D7D2A2D2D2D2D3A2D2D2D2D2A2A0A2D2D2D2D2A2A0A2D2D2D2D2A2A0A2D2A2A0A2A2A2A2D2D2D2D0A2A2A2A2D2D2D2D2A0A0A2D2A2A2A2D2D2D2D0A0A2A2A2A2D2D2D2D4D4A4A3A5D3A5D3A3A3D4A5D3D4A3D0A2A0A2D2A2A7A2D5A0A5D4A2D2D2D2D3A2D2D2D2D2A2A0A2D2D2D2D2A2A0A2D2D2D2D2A2A0A2D2A2A2D3D6A3D0A2A2A2A2D2D2D2D3A5D3A2D4A7D5A3D4A3D6A2D2D2D2D3A3A3A2D3D2D2D2A2A5D7A2D2A2A2A0A2D2A3D5D3A5D3A5D0A0A5D3A5D3A5D0A4D3D6A2A2A2D2D2D2D2D2D2A3D2D2D2A5A2D2D2D2D2D2D6A2D2D2D2D2D2D2D2A5A2A2A2D2D2A2A3A7D3A2A

3D4A7D5A3D4A3D3D3D6A7D2A2D2D2D2D3A2D2D2D2D2A2A0A2D2D2D2D2A2A0A2D2D2D2D2A2A0A2D2A2A0A2A2A2A2D

In [544]:
# Decode patterns
decoded_patterns, decoded_patterns_combined, decoded_patterns_count = decode_results(finalpat, count_pattern, combined, beforereduction)
    


for pattern in decoded_patterns:
        print("Pattern:", pattern)
        print("importance score:", decoded_patterns_combined[pattern],
              "occurrence:", decoded_patterns_count[pattern])


Pattern: 2A2A0A2D2A2A
importance score: 19 occurrence: 13
Pattern: 2D2D2A2A
importance score: 46 occurrence: 29
Pattern: 2D2D2D
importance score: 103 occurrence: 61
Pattern: 2A2A2D2D
importance score: 33 occurrence: 22
Pattern: 2D2D2D2D
importance score: 85 occurrence: 51
Pattern: 2D2A2A2A
importance score: 18 occurrence: 16
Pattern: 2A2A2A
importance score: 46 occurrence: 32


In [545]:
decoded_patterns

['2A2A0A2D2A2A',
 '2D2D2A2A',
 '2D2D2D',
 '2A2A2D2D',
 '2D2D2D2D',
 '2D2A2A2A',
 '2A2A2A']

In [546]:
# Get patterns from ground truth

dict_wordtonum = {"Unison": '2', "Second": '2', "Third": '3', "Fourth": '4', "Fifth": '5', "Sixth": '6', "Seventh": '7'}
    
groundtruthpatterns = []
print(glob.glob("jkupddtest/chopinmono/*"))
filenames = glob.glob("jkupddtest/chopinmono/B/*.csv")
for filename in filenames:
    print(filename)
    with open(filename, 'r') as file:
        csvreader = csv.reader(file)
        gts = []
        gtm = []
        pattern_gt = []
        for row in csvreader:
            gts.append(float(row[0]))
            gtm.append(float(row[1]))
        for i in gtm:
            pattern_gt.append(m21.note.Note(i))
        pattern_string = ""
        interval_string = ""
        for i in pattern_gt:
            pattern_string += str(i)
            
        for i in range(1, len(pattern_gt)):           
            gap = pattern_gt[i].pitch.diatonicNoteNum - pattern_gt[i-1].pitch.diatonicNoteNum
            if gap > 0:
                direction = 'A'
            else:
                direction = 'D'
            m21_interval_directed = m21.interval.Interval(noteStart=pattern_gt[i-1], noteEnd=pattern_gt[i]).directedSimpleNiceName
            arr_diatonic = m21_interval_directed.split(" ")
            m21_generic = dict_wordtonum[arr_diatonic[-1]]
            m21_interval = m21_generic+direction
            
            interval_string+=m21_interval
            
        print(interval_string)

        groundtruthpatterns.append(interval_string)


['jkupddtest/chopinmono/A', 'jkupddtest/chopinmono/mazurka24-4.krn', 'jkupddtest/chopinmono/B']
jkupddtest/chopinmono/B/occ5.csv
3D3A3D2D2A2D2D2A2D5A4D4A4D2D2D2D2D3A3D5A3D3A3D2D2A2D2A2A2D2D6D2A
jkupddtest/chopinmono/B/occ1.csv
3D3A3D2D2A2D2D5A4D4A4D2D2D2D2D5A3D3A3D2D2A2D2A2A2D2D6D2A
jkupddtest/chopinmono/B/occ7.csv
3D3A3D2D2A2D2D2A2D5A4D4A4D2D2D2D2D3A3D5A3D3A3D2D2A2D2A2A2D2D6D2A
jkupddtest/chopinmono/B/occ2.csv
3D3A3D2D2A2D2D5A4D4A4D2D2D2D2D5A3D3A3D2D2A2D2A2D2D4D2D2D
jkupddtest/chopinmono/B/occ6.csv
3D3A3D2D2A2D2D5A4D4A4D2D2D2D2D5A3D3A3D2D2A2D2A2D2D4D2D2D
jkupddtest/chopinmono/B/occ4.csv
3D3A3D2D2A2D2D5A4D4A4D2D2D2D2D5A3D3A3D2D2A2D2A2D2D4D2D2D
jkupddtest/chopinmono/B/occ3.csv
3D3A3D2D2A2D2D2A2D5A4D4A4D2D2D2D2D3A3D5A3D3A3D2D2A2D2A2A2D2D6D2A


In [547]:
set(groundtruthpatterns)

{'3D3A3D2D2A2D2D2A2D5A4D4A4D2D2D2D2D3A3D5A3D3A3D2D2A2D2A2A2D2D6D2A',
 '3D3A3D2D2A2D2D5A4D4A4D2D2D2D2D5A3D3A3D2D2A2D2A2A2D2D6D2A',
 '3D3A3D2D2A2D2D5A4D4A4D2D2D2D2D5A3D3A3D2D2A2D2A2D2D4D2D2D'}

In [517]:
for pattern in groundtruthpatterns:
    count = 0
    for decodedstring in decoded_strings:
        count += decodedstring.count(pattern)
    if count == 0:
        print("pattern", pattern, "not in original score")
    elif count == 1:
        print("pattern", pattern, "only appeared once")
    else:
        print("pattern", pattern, "appeared", count, "times in total")

pattern 3D3A3D2D2A2D2D2A2D5A4D4A4D2D2D2D2D3A3D5A3D3A3D2D2A2D2A2A2D2D6D2A not in original score
pattern 3D3A3D2D2A2D2D5A4D4A4D2D2D2D2D5A3D3A3D2D2A2D2A2A2D2D6D2A not in original score
pattern 3D3A3D2D2A2D2D2A2D5A4D4A4D2D2D2D2D3A3D5A3D3A3D2D2A2D2A2A2D2D6D2A not in original score
pattern 3D3A3D2D2A2D2D5A4D4A4D2D2D2D2D5A3D3A3D2D2A2D2A2D2D4D2D2D not in original score
pattern 3D3A3D2D2A2D2D5A4D4A4D2D2D2D2D5A3D3A3D2D2A2D2A2D2D4D2D2D not in original score
pattern 3D3A3D2D2A2D2D5A4D4A4D2D2D2D2D5A3D3A3D2D2A2D2A2D2D4D2D2D not in original score
pattern 3D3A3D2D2A2D2D2A2D5A4D4A4D2D2D2D2D3A3D5A3D3A3D2D2A2D2A2A2D2D6D2A not in original score


In [333]:
##CHECK IF THERE IS A MATCH

for i in groundtruthpatterns:
    if i in finalpat: #refined_finalpat:
        print("found a match:", i)

In [263]:
#TEST2


In [270]:
#groundtruthpath1 = "jkupddtest/gibbonsmono/silverswan.krn"
import pandas as pd
gtfiles = glob.glob("jkupddtest/bachmono/B/midi/*.csv")
print(gtfiles)
gtstrings = []
for gtfile in gtfiles:
    print(gtfile)
    with open(gtfile, 'r') as file:
        df = pd.read_csv(file)
        #print(df)
        #for midi_note_num in df["midi_note_num"]:
        #    print(midi_note_num)
        count = 0
        gtstring = ""
        for diatonic in df["diatonic_interval"]:
            if count == 0:
                # ignore the first "interval" number which is always 0
                count+=1
                continue
            count+=1
            # just to format the input
            if diatonic == -1:
                diatonic = -2
            if diatonic == 1:
                diatonic = 2
            if diatonic < 0:
                dia = str(abs(diatonic)) + 'D'
            else:
                dia = str(abs(diatonic)) + 'A'
            gtstring += dia
        gtstrings.append(gtstring)
        print(gtstring)
            

['jkupddtest/bachmono/B/midi/occ1.midi_part1.csv', 'jkupddtest/bachmono/B/midi/occ2.midi_part1.csv', 'jkupddtest/bachmono/B/midi/occ4.midi_part1.csv', 'jkupddtest/bachmono/B/midi/occ7.midi_part1.csv', 'jkupddtest/bachmono/B/midi/occ3.midi_part1.csv', 'jkupddtest/bachmono/B/midi/occ6.midi_part1.csv', 'jkupddtest/bachmono/B/midi/occ5.midi_part1.csv']
jkupddtest/bachmono/B/midi/occ1.midi_part1.csv
2D2D2D2D2A2D2D2D2D2A2A2D2D2D2D2A2A2D2D2D2D2A2A0A2D2A2A
jkupddtest/bachmono/B/midi/occ2.midi_part1.csv
2D2D2D2D2A2D2D2D2D2A2A2D2D2D2D2A2A2D2D2D2D2A2A0A2D2A2A
jkupddtest/bachmono/B/midi/occ4.midi_part1.csv
2D2D2D2D2A2D2D2D2D2A2A2D2D2D0A0A2A2D0A2D2D2A2A0A2D2A0A
jkupddtest/bachmono/B/midi/occ7.midi_part1.csv
2A2A2D2D2D2D2A2A2D2D2D2D2A2A0A2D2A2A
jkupddtest/bachmono/B/midi/occ3.midi_part1.csv
2D2D2D2D2A2D2D2D2D2A2A2D2D2D0A0A2A2D0A2D2D2A2A0A2D2A0A
jkupddtest/bachmono/B/midi/occ6.midi_part1.csv
2D2D2D2D2A2D2D2D2D2A2A2D2D2D2D2A2A2D2D2D2D2A2A0A2D2A2A
jkupddtest/bachmono/B/midi/occ5.midi_part1.csv
2D2D2D2D

In [271]:
print(set(gtstrings))

{'2D2D2D2D2A2D2D2D2D2A2A2D2D2D0A0A2A2D0A2D2D2A2A0A2D2A0A', '2D2D2D2D2A2D2D2D2D2A2A2D2D2D2D2A2A2D2D2D2D2A2A0A2D2A2A', '2A2A2D2D2D2D2A2A2D2D2D2D2A2A0A2D2A2A'}


In [272]:
for pattern in gtstrings:
    count = 0
    for decodedstring in decoded_strings:
        count += decodedstring.count(pattern)
    if count == 0:
        print("pattern", pattern, "not in original score")
    elif count == 1:
        print("pattern", pattern, "only appeared once")
    else:
        print("pattern", pattern, "appeared", count, "times in total")

pattern 2D2D2D2D2A2D2D2D2D2A2A2D2D2D2D2A2A2D2D2D2D2A2A0A2D2A2A not in original score
pattern 2D2D2D2D2A2D2D2D2D2A2A2D2D2D2D2A2A2D2D2D2D2A2A0A2D2A2A not in original score
pattern 2D2D2D2D2A2D2D2D2D2A2A2D2D2D0A0A2A2D0A2D2D2A2A0A2D2A0A not in original score
pattern 2A2A2D2D2D2D2A2A2D2D2D2D2A2A0A2D2A2A not in original score
pattern 2D2D2D2D2A2D2D2D2D2A2A2D2D2D0A0A2A2D0A2D2D2A2A0A2D2A0A not in original score
pattern 2D2D2D2D2A2D2D2D2D2A2A2D2D2D2D2A2A2D2D2D2D2A2A0A2D2A2A not in original score
pattern 2D2D2D2D2A2D2D2D2D2A2A2D2D2D2D2A2A2D2D2D2D2A2A0A2D2A2A not in original score
